In [ ]:
# %% [markdown]
# # 城市功能区分析 - 可视化展示
# 
# 作者: 张笔弈
# 日期: 2026-01-20
# 
# 本笔记本展示可视化分析的完整流程

# %%
import sys
sys.path.append("..")

import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yaml

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# %%
# 加载配置
with open("../config/settings.yaml", "r") as f:
    config = yaml.safe_load(f)

# %%
# 加载数据
zones_data = gpd.read_file("../data/processed/function_zones.geojson")
poi_data = gpd.read_file("../data/processed/guangzhou_pois.geojson")
grid_data = gpd.read_file("../data/processed/integrated_grid.geojson")

print(f"功能区数据: {len(zones_data)}个网格")
print(f"POI数据: {len(poi_data)}条记录")
print(f"功能区类型分布:")
print(zones_data['zone_type'].value_counts())

# %%
# 1. 静态可视化

# 1.1 功能区分布图
plt.figure(figsize=(14, 10))

# 计算中心点
zones_data['centroid_lon'] = zones_data.geometry.centroid.x
zones_data['centroid_lat'] = zones_data.geometry.centroid.y

# 为每种功能区分配颜色
zone_types = zones_data['zone_type'].unique()
colors = plt.cm.Set3(np.linspace(0, 1, len(zone_types)))
color_map = dict(zip(zone_types, colors))

# 绘制散点图（简化，实际应该显示多边形）
for zone_type, color in color_map.items():
    zone_subset = zones_data[zones_data['zone_type'] == zone_type]
    plt.scatter(zone_subset['centroid_lon'], zone_subset['centroid_lat'], 
               c=[color], label=zone_type, s=50, alpha=0.7, edgecolors='black')

plt.xlabel('经度')
plt.ylabel('纬度')
plt.title('城市功能区分布')
plt.legend(title='功能区类型', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("../output/function_zones_distribution.png", dpi=300, bbox_inches='tight')
plt.show()

# %%
# 1.2 功能区统计图
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 功能区数量分布
zone_counts = zones_data['zone_type'].value_counts()
axes[0, 0].bar(zone_counts.index, zone_counts.values, color=[color_map[z] for z in zone_counts.index])
axes[0, 0].set_title('功能区数量分布')
axes[0, 0].set_xlabel('功能区类型')
axes[0, 0].set_ylabel('网格数量')
axes[0, 0].tick_params(axis='x', rotation=45)

# POI密度分布
zone_poi_stats = zones_data.groupby('zone_type')['poi_count'].agg(['mean', 'std']).sort_values('mean', ascending=False)
axes[0, 1].bar(zone_poi_stats.index, zone_poi_stats['mean'], 
               yerr=zone_poi_stats['std'], capsize=5, 
               color=[color_map[z] for z in zone_poi_stats.index])
axes[0, 1].set_title('各功能区平均POI密度')
axes[0, 1].set_xlabel('功能区类型')
axes[0, 1].set_ylabel('平均POI数量')
axes[0, 1].tick_params(axis='x', rotation=45)

# NDVI分布
if 'ndvi_mean' in zones_data.columns:
    zone_ndvi_stats = zones_data.groupby('zone_type')['ndvi_mean'].agg(['mean', 'std']).sort_values('mean', ascending=False)
    axes[1, 0].bar(zone_ndvi_stats.index, zone_ndvi_stats['mean'], 
                   yerr=zone_ndvi_stats['std'], capsize=5,
                   color=[color_map[z] for z in zone_ndvi_stats.index])
    axes[1, 0].set_title('各功能区平均植被指数(NDVI)')
    axes[1, 0].set_xlabel('功能区类型')
    axes[1, 0].set_ylabel('平均NDVI')
    axes[1, 0].tick_params(axis='x', rotation=45)
else:
    axes[1, 0].text(0.5, 0.5, 'NDVI数据不可用', ha='center', va='center', fontsize=12)
    axes[1, 0].set_title('NDVI分布')

# 路网密度分布
if 'road_density' in zones_data.columns:
    zone_road_stats = zones_data.groupby('zone_type')['road_density'].agg(['mean', 'std']).sort_values('mean', ascending=False)
    axes[1, 1].bar(zone_road_stats.index, zone_road_stats['mean'], 
                   yerr=zone_road_stats['std'], capsize=5,
                   color=[color_map[z] for z in zone_road_stats.index])
    axes[1, 1].set_title('各功能区平均路网密度')
    axes[1, 1].set_xlabel('功能区类型')
    axes[1, 1].set_ylabel('路网密度')
    axes[1, 1].tick_params(axis='x', rotation=45)
else:
    axes[1, 1].text(0.5, 0.5, '路网密度数据不可用', ha='center', va='center', fontsize=12)
    axes[1, 1].set_title('路网密度分布')

plt.tight_layout()
plt.savefig("../output/function_zones_statistics.png", dpi=300, bbox_inches='tight')
plt.show()

# %%
# 1.3 特征相关性热力图
plt.figure(figsize=(12, 10))

# 选择数值特征
numeric_features = zones_data.select_dtypes(include=[np.number]).columns
if len(numeric_features) > 1:
    correlation_matrix = zones_data[numeric_features].corr()
    
    sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', 
                center=0, square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
    plt.title('功能区特征相关性矩阵')
    plt.tight_layout()
    plt.savefig("../output/zones_feature_correlation.png", dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("数值特征不足，无法计算相关性矩阵")

# %%
# 2. 交互式可视化

# 2.1 使用Plotly创建交互式散点图
if 'ndvi_mean' in zones_data.columns and 'road_density' in zones_data.columns:
    fig = px.scatter(zones_data, 
                    x='poi_count', 
                    y='ndvi_mean',
                    color='zone_type',
                    size='road_density',
                    hover_data=['grid_id', 'poi_diversity'],
                    title='功能区特征分布（交互式）',
                    labels={
                        'poi_count': 'POI数量',
                        'ndvi_mean': '植被指数',
                        'zone_type': '功能区类型',
                        'road_density': '路网密度'
                    },
                    color_discrete_map=color_map)
    
    fig.update_layout(
        xaxis_title="POI数量",
        yaxis_title="植被指数(NDVI)",
        legend_title="功能区类型",
        hovermode='closest'
    )
    
    fig.write_html("../output/interactive_scatter_plot.html")
    print("交互式散点图已保存: ../output/interactive_scatter_plot.html")
    
    # 在notebook中显示
    fig.show()

# %%
# 2.2 创建3D散点图
fig_3d = go.Figure(data=[go.Scatter3d(
    x=zones_data['poi_count'],
    y=zones_data['ndvi_mean'] if 'ndvi_mean' in zones_data.columns else np.zeros(len(zones_data)),
    z=zones_data['road_density'] if 'road_density' in zones_data.columns else np.zeros(len(zones_data)),
    mode='markers',
    marker=dict(
        size=8,
        color=[list(color_map[z])[:3] for z in zones_data['zone_type']],  # 转换为RGB
        opacity=0.8
    ),
    text=zones_data['zone_type'],
    hoverinfo='text'
)])

fig_3d.update_layout(
    title='功能区特征3D分布',
    scene=dict(
        xaxis_title='POI数量',
        yaxis_title='植被指数',
        zaxis_title='路网密度'
    ),
    width=1000,
    height=800
)

fig_3d.write_html("../output/3d_scatter_plot.html")
print("3D散点图已保存: ../output/3d_scatter_plot.html")

# %%
# 2.3 使用Folium创建交互式地图
print("创建交互式地图...")

# 计算地图中心点
center_lat = zones_data['centroid_lat'].mean()
center_lon = zones_data['centroid_lon'].mean()

# 创建基础地图
m = folium.Map(location=[center_lat, center_lon], zoom_start=13, tiles='CartoDB positron')

# 添加功能区图层
for _, row in zones_data.iterrows():
    # 获取几何中心点
    centroid = row['geometry'].centroid
    
    # 创建弹出窗口内容
    popup_content = f"""
    <div style="font-family: Arial, sans-serif; font-size: 12px;">
        <b>网格ID:</b> {row['grid_id']}<br>
        <b>功能区类型:</b> {row['zone_type']}<br>
        <b>POI数量:</b> {row.get('poi_count', 'N/A')}<br>
        <b>植被指数:</b> {row.get('ndvi_mean', 'N/A'):.2f}<br>
        <b>路网密度:</b> {row.get('road_density', 'N/A'):.4f}<br>
        <b>多样性指数:</b> {row.get('poi_diversity', 'N/A'):.2f}
    </div>
    """
    
    # 根据功能区类型选择颜色
    zone_color = '#{:02x}{:02x}{:02x}'.format(
        int(color_map[row['zone_type']][0] * 255),
        int(color_map[row['zone_type']][1] * 255),
        int(color_map[row['zone_type']][2] * 255)
    )
    
    # 添加标记
    folium.CircleMarker(
        location=[centroid.y, centroid.x],
        radius=8,
        popup=popup_content,
        color=zone_color,
        fill=True,
        fill_color=zone_color,
        fill_opacity=0.7,
        weight=1
    ).add_to(m)

# 添加POI热点图层（示例）
if len(poi_data) > 0:
    # 随机选择部分POI显示
    poi_sample = poi_data.sample(min(100, len(poi_data)))
    
    heat_data = [[row.geometry.y, row.geometry.x, 1] for _, row in poi_sample.iterrows()]
    
    plugins.HeatMap(heat_data, radius=15, blur=10, max_zoom=1).add_to(m)

# 添加图层控制
folium.LayerControl().add_to(m)

# 添加全屏控件
plugins.Fullscreen().add_to(m)

# 保存地图
map_path = "../output/interactive_map.html"
m.save(map_path)

print(f"交互式地图已保存: {map_path}")

# %%
# 3. 高级可视化

# 3.1 雷达图 - 各功能区特征对比
fig_radar = go.Figure()

# 选择要展示的特征
radar_features = ['poi_count', 'poi_diversity', 'ndvi_mean', 'road_density']
available_radar_features = [f for f in radar_features if f in zones_data.columns]

if len(available_radar_features) >= 3:
    for zone_type in zones_data['zone_type'].unique():
        zone_data = zones_data[zones_data['zone_type'] == zone_type]
        
        # 计算特征平均值
        feature_means = []
        for feature in available_radar_features:
            feature_means.append(zone_data[feature].mean())
        
        # 归一化
        feature_means_norm = []
        for i, feature in enumerate(available_radar_features):
            max_val = zones_data[feature].max()
            min_val = zones_data[feature].min()
            if max_val > min_val:
                norm_val = (feature_means[i] - min_val) / (max_val - min_val)
            else:
                norm_val = 0.5
            feature_means_norm.append(norm_val)
        
        # 添加雷达图轨迹
        fig_radar.add_trace(go.Scatterpolar(
            r=feature_means_norm + [feature_means_norm[0]],  # 闭合图形
            theta=available_radar_features + [available_radar_features[0]],
            fill='toself',
            name=zone_type,
            line_color=f'rgb({int(color_map[zone_type][0]*255)}, {int(color_map[zone_type][1]*255)}, {int(color_map[zone_type][2]*255)})'
        ))
    
    fig_radar.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 1]
            )
        ),
        title='各功能区特征雷达图对比',
        showlegend=True,
        width=800,
        height=600
    )
    
    fig_radar.write_html("../output/radar_chart.html")
    print("雷达图已保存: ../output/radar_chart.html")
    
else:
    print("可用特征不足，无法创建雷达图")

# %%
# 3.2 时间序列变化图（模拟数据）
# 创建模拟时间序列数据
np.random.seed(42)
time_points = pd.date_range('2023-01-01', '2023-12-31', freq='M')
simulated_changes = []

for zone_type in zones_data['zone_type'].unique():
    base_poi = zones_data[zones_data['zone_type'] == zone_type]['poi_count'].mean()
    
    for timestamp in time_points:
        # 模拟增长趋势
        trend = 0.02 * (time_points.get_loc(timestamp) + 1)
        noise = np.random.normal(0, 0.1)
        
        simulated_changes.append({
            'timestamp': timestamp,
            'zone_type': zone_type,
            'poi_count': max(0, base_poi * (1 + trend + noise)),
            'change_rate': trend + noise
        })

changes_df = pd.DataFrame(simulated_changes)

# 创建时间序列图
fig_timeseries = make_subplots(
    rows=2, cols=1,
    subplot_titles=('各功能区POI数量变化', '各功能区变化率'),
    vertical_spacing=0.1
)

for zone_type in changes_df['zone_type'].unique():
    zone_data = changes_df[changes_df['zone_type'] == zone_type]
    
    # POI数量变化
    fig_timeseries.add_trace(
        go.Scatter(
            x=zone_data['timestamp'],
            y=zone_data['poi_count'],
            mode='lines+markers',
            name=zone_type,
            line=dict(color=f'rgb({int(color_map[zone_type][0]*255)}, {int(color_map[zone_type][1]*255)}, {int(color_map[zone_type][2]*255)})')
        ),
        row=1, col=1
    )
    
    # 变化率
    fig_timeseries.add_trace(
        go.Scatter(
            x=zone_data['timestamp'],
            y=zone_data['change_rate'],
            mode='lines',
            name=zone_type,
            showlegend=False,
            line=dict(color=f'rgb({int(color_map[zone_type][0]*255)}, {int(color_map[zone_type][1]*255)}, {int(color_map[zone_type][2]*255)})',
                     dash='dash')
        ),
        row=2, col=1
    )

fig_timeseries.update_xaxes(title_text="时间", row=2, col=1)
fig_timeseries.update_yaxes(title_text="POI数量", row=1, col=1)
fig_timeseries.update_yaxes(title_text="变化率", row=2, col=1)

fig_timeseries.update_layout(
    height=800,
    title_text="功能区时间序列变化分析",
    hovermode='x unified'
)

fig_timeseries.write_html("../output/time_series_analysis.html")
print("时间序列分析图已保存: ../output/time_series_analysis.html")

# %%
# 4. 导出可视化报告
report_content = f"""# 城市功能区可视化分析报告

**生成时间**: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}

## 分析概览
- **分析区域**: {config.get('region', {}).get('name', '未指定')}
- **功能区数量**: {len(zones_data)}个网格
- **功能区类型**: {len(zone_types)}种
- **POI数据量**: {len(poi_data)}条记录

## 主要发现
1. **功能区分布特征**: {zone_counts.idxmax()}类功能区数量最多
2. **POI密度差异**: 各类功能区POI密度存在显著差异
3. **空间分布规律**: 功能区呈现一定的空间聚集特征

## 可视化成果
1. **静态图表**: 包含分布图、统计图、相关性矩阵等
2. **交互式地图**: 支持点击查看详细信息
3. **3D可视化**: 展示多维特征关系
4. **时间序列分析**: 展示变化趋势

## 建议应用
1. 城市规划: 基于功能区分布优化城市布局
2. 商业分析: 识别商业潜力区域
3. 环境评估: 评估城市生态环境质量
4. 交通规划: 优化路网布局

## 文件清单
- 功能区分布图: `function_zones_distribution.png`
- 功能区统计图: `function_zones_statistics.png`
- 交互式地图: `interactive_map.html`
- 3D散点图: `3d_scatter_plot.html`
- 时间序列分析: `time_series_analysis.html`
"""

with open("../output/visualization_report.md", "w", encoding="utf-8") as f:
    f.write(report_content)

print("\n可视化分析完成!")
print("报告已保存: ../output/visualization_report.md")